In [4]:
from lda import lda
from lda import load_docs_in_bow, process_vectors, average_document_length, assign_topic, get_word_assignment_info, get_document_assignment_info
import numpy as np
from collections import Counter

In [5]:
N_TOPIC = 10
EPOCHS = 200
ALPHA = 0.5
BETA = 0.01

In [6]:
path_to_doc = "../data/docs_trec_covid"

vocabulary, vectors, doc_id = load_docs_in_bow(path_to_doc, n_doc=10)

dict_document = process_vectors(vectors)

In [7]:
print(len(vocabulary))
print(average_document_length(dict_document))

551
73.1


In [8]:
def int2string(dict_document, doc_id, vocabulary):
    new_dict_document = {}
    
    for document, word_topic in dict_document.items():
        new_dict_document[doc_id[document]] = list(map(lambda x: (vocabulary[x[0]], x[1]), word_topic))
        
    return new_dict_document

In [9]:
topic_assignment = assign_topic(dict_document, N_TOPIC)

In [10]:
topic_assignment = lda(topic_assignment, N_TOPIC, len(vocabulary), EPOCHS, ALPHA, BETA)

	Starting 1 iteration...

		 sampling 0 document...
		 sampling 1 document...
		 sampling 2 document...
		 sampling 3 document...
		 sampling 4 document...
		 sampling 5 document...
		 sampling 6 document...
		 sampling 7 document...
		 sampling 8 document...
		 sampling 9 document...
	Starting 2 iteration...

		 sampling 0 document...
		 sampling 1 document...
		 sampling 2 document...
		 sampling 3 document...
		 sampling 4 document...
		 sampling 5 document...
		 sampling 6 document...
		 sampling 7 document...
		 sampling 8 document...
		 sampling 9 document...
	Starting 3 iteration...

		 sampling 0 document...
		 sampling 1 document...
		 sampling 2 document...
		 sampling 3 document...
		 sampling 4 document...
		 sampling 5 document...
		 sampling 6 document...
		 sampling 7 document...
		 sampling 8 document...
		 sampling 9 document...
	Starting 4 iteration...

		 sampling 0 document...
		 sampling 1 document...
		 sampling 2 document...
		 sampling 3 document...
		 sampling 

In [9]:
# def get_word_wise_assignment(topic_assignment, vocabulary, n_topics, n_vocabularies):
#     matrix_word_topic = np.zeros((n_vocabularies, n_topics))
#     for word_topic in topic_assignment.values():
#         for word, topic in word_topic:
#             matrix_word_topic[word, topic] += 1
# 
#     return matrix_word_topic
# 
# 
# def get_word_assignment_info(topic_assignment, vocabulary, n_topics, n_vocabularies, top_topic=5, top_word=10):
#     word_topic_matrix = get_word_wise_assignment(topic_assignment, vocabulary, n_topics, n_vocabularies)
#     top_topic_index = np.argsort(word_topic_matrix.sum(axis=0))[::-1]
#     topic_percentage = np.sort(word_topic_matrix.sum(axis=0) / word_topic_matrix.sum())[::-1]
#     print(f'------- The top {top_topic} topics-------\n')
#     print(f'It\'s top {top_word} words:')
#     for t in range(top_topic):
#         print(f'- topic {top_topic_index[t]} ({topic_percentage[t] * 100 :.2f}%)')
#         word_assigment_vector = word_topic_matrix[:, top_topic_index[t]]
#         top_word_index = np.argsort(word_assigment_vector)[::-1]
#         word_percentage = np.sort(word_assigment_vector / word_assigment_vector.sum())[::-1]
#         for w in range(top_word):
#             print(f'\t- {vocabulary[top_word_index[w]]} ({word_percentage[w] * 100:.2f}%)')

In [8]:
get_word_assignment_info(topic_assignment, vocabulary, N_TOPIC, len(vocabulary))

------- The top 5 topics -------

It's top 10 words:
- topic 2 (12.04%)
	- alter (2.27%)
	- mice (2.27%)
	- human (2.27%)
	- specif (2.27%)
	- cell (2.27%)
	- respiratori (2.27%)
	- virus (2.27%)
	- epith (2.27%)
	- respon (2.27%)
	- gene (1.14%)
- topic 5 (11.63%)
	- hospit (2.35%)
	- spatial (1.18%)
	- categori (1.18%)
	- evalu (1.18%)
	- rod (1.18%)
	- classifi (1.18%)
	- sale (1.18%)
	- new (1.18%)
	- network (1.18%)
	- send (1.18%)
- topic 0 (11.49%)
	- disea (4.76%)
	- inflammatori (4.76%)
	- lung (3.57%)
	- protein (3.57%)
	- pulmonari (3.57%)
	- oxid (3.57%)
	- activ (3.57%)
	- contribut (3.57%)
	- mediat (3.57%)
	- associ (2.38%)
- topic 7 (10.12%)
	- although (2.70%)
	- patient (2.70%)
	- mortal (2.70%)
	- abdulaziz (1.35%)
	- round (1.35%)
	- decemb (1.35%)
	- data (1.35%)
	- recov (1.35%)
	- recoveri (1.35%)
	- cultur (1.35%)
- topic 6 (9.71%)
	- mechan (2.82%)
	- studi (2.82%)
	- regul (2.82%)
	- contain (2.82%)
	- yield (1.41%)
	- collagen (1.41%)
	- depend (1.41%)
	- dem

In [11]:
# def get_document_wise_assignment(topic_assignment, n_topics):
#     n_documents = len(topic_assignment)
#     matrix_document_topic = np.zeros((n_documents, n_topics))
#     for document, word_topic in topic_assignment.items():
#         for _, topic in word_topic:
#             matrix_document_topic[document, topic] += 1
# 
#     return matrix_document_topic
# 
# 
# def get_document_assignment_info(topic_assignment, doc_id, n_topics, n_documents=5):
#     document_topic_matrix = get_document_wise_assignment(topic_assignment, n_topics)
#     print('The distribution of topics to documents:')
#     for d in range(n_documents):
#         document_topic_vector = document_topic_matrix[d]
#         top_topic_index = np.argsort(document_topic_vector)[::-1]
#         topic_percentage = np.sort(document_topic_vector / document_topic_vector.sum())[::-1]
#         string_topic = f'{topic_percentage[0]:.3f} * topic{top_topic_index[0]}'
#         
#         for topic, percentage in zip(top_topic_index[1:], topic_percentage[1:]):
#             if percentage <= 0.0001:
#                 break
#             else:
#                 string_topic += ' + '
#                 string_topic += f'{percentage:.3f} * topic{topic}'
#                 
#         print(f'\t- {doc_id[d]} : {string_topic}')

In [9]:
get_document_assignment_info(topic_assignment, doc_id, N_TOPIC)

The distribution of topics to documents:
	- ug7v899j : 0.663 * topic7 + 0.225 * topic1 + 0.101 * topic9 + 0.011 * topic5
	- 02tnwd4m : 0.515 * topic9 + 0.394 * topic0 + 0.045 * topic8 + 0.030 * topic7 + 0.015 * topic6
	- ejv2xln0 : 0.526 * topic2 + 0.351 * topic0 + 0.041 * topic8 + 0.041 * topic6 + 0.031 * topic1 + 0.010 * topic4
	- 2b73a28n : 0.484 * topic3 + 0.452 * topic0 + 0.032 * topic6 + 0.032 * topic4
	- 9785vg6d : 0.569 * topic2 + 0.231 * topic1 + 0.138 * topic8 + 0.062 * topic3


In [ ]:
from query_ranking import get_term_probability, compute_cf, query_score

In [11]:
cf_counter = compute_cf(dict_document)
all_cf = 0
for i in range(len(vocabulary)):
    all_cf += cf_counter[i]

In [12]:
query = ['antigen', 'statist']
int_query = list(map(lambda x: list(vocabulary).index(x), query))

In [13]:
scores = query_score(int_query, dict_document, topic_assignment, N_TOPIC, len(vocabulary), ALPHA, BETA)

In [14]:
print(scores)

[-13.887595835612643, -13.909539610134193, -12.895448470641885, -13.889335951889858, -13.068566322828952, -13.915724319210426, -13.900438677165734, -13.888463227740932, -13.933302970853514, -12.491280589258551]
